### 本次作業嘗試調整學習率的設定方式，從過往的定值設定改成使用adam讓模型訓練過程自主調整。
### 並且把資料切割出訓練集跟驗證集做觀察，最後模型正確率約9成在可接受範圍

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## 1. 讀入數據集

In [2]:
from tensorflow.keras.datasets import fashion_mnist

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

## 2. 查看數據集的內容，確認資料型態

In [4]:
x_train.shape

(60000, 28, 28)

In [5]:
x_test.shape

(10000, 28, 28)

## 3. 資料處理

### 資料正規化  (0 , 255) --> (0 , 1)

In [6]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255
x_test = x_test.reshape(10000, 28, 28, 1) / 255

### 數據集資料為灰階，即只有一個channel，轉換資料格式從(28 , 28) --> (28 , 28 , 1)

In [7]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

### 對輸出資料做 one-hot encoding

In [8]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## 4.建造捲積神經網路 CNN

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout 
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras.optimizers import SGD


In [10]:
model = Sequential()

### 建立第一層Conv2D 與 MaxPooling2D，設計16個Filter和3*3的篩選機制

In [11]:
model.add(Conv2D(16,(3,3), padding='same',input_shape = (28,28,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

### 建立第二層Conv2D 與 MaxPooling2D，設計32個Filter和3*3的篩選機制

In [12]:
model.add(Conv2D(32,(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

### 建立第三層Conv2D 與 MaxPooling2D，設計64個Filter和6*6的篩選機制

In [13]:
model.add(Conv2D(64,(6,6), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [14]:
model.add(Flatten())

In [15]:
model.add(Dense(60, activation='relu'))

In [16]:
model.add(Dense(10 ,activation='softmax'))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

## 5. 訓練模型

### optimizer 試著使用adam，看看訓練過程自己調整學習率的效果如何

In [18]:
model.compile(loss = 'mse', optimizer = , metrics=['accuracy'])

### 兩成訓練資料當成驗證集

In [19]:
model.fit(x_train, y_train, batch_size=64, epochs=16 , validation_split = 0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/16
48000/48000 [==============================] - 8s 167us/sample - loss: 0.0273 - accuracy: 0.8044 - val_loss: 0.0202 - val_accuracy: 0.8602
Epoch 2/16
48000/48000 [==============================] - 6s 117us/sample - loss: 0.0172 - accuracy: 0.8827 - val_loss: 0.0167 - val_accuracy: 0.8847
Epoch 3/16
48000/48000 [==============================] - 6s 123us/sample - loss: 0.0147 - accuracy: 0.8994 - val_loss: 0.0142 - val_accuracy: 0.9043
Epoch 4/16
48000/48000 [==============================] - 6s 122us/sample - loss: 0.0131 - accuracy: 0.9117 - val_loss: 0.0140 - val_accuracy: 0.9059
Epoch 5/16
48000/48000 [==============================] - 6s 115us/sample - loss: 0.0119 - accuracy: 0.9215 - val_loss: 0.0131 - val_accuracy: 0.9097
Epoch 6/16
48000/48000 [==============================] - 6s 115us/sample - loss: 0.0108 - accuracy: 0.9281 - val_loss: 0.0130 - val_accuracy: 0.9109
Epoch 7/16
48000/48000 [==========================

## 6. 訓練成果

In [115]:
score = model.evaluate(x_test,y_test)
loss, acc = score
print('資料正確率為:', acc)

10000/10000 [==============================] - 2s 230us/sample - loss: 0.0159 - acc: 0.9052
資料正確率為: 0.9052


In [116]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [117]:
result = model.predict_classes(x_test)

In [118]:
def mypredict(n):
    print('CNN預測結果為:', class_names[result[n]])
    X = x_test[n].reshape(28,28)
    plt.imshow(X,cmap='Greys');

In [119]:
from ipywidgets import interact_manual

In [120]:
interact_manual(mypredict, n=(0,9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

In [121]:
model.save('myCNNmodel.h5')